# Segmenting and Clustering Neighborhoods in Toronto

by Jianxu Shi   
Data Scientist in Training

*This notebook is created to complete the second peer-graded assignment in course "Applied Data Science Capstone". 
The goal is to segment and cluster the neighborhoods in the city of Toronta. The process consists of five steps.
First, a list of postal codes of Canada, along with associated boroughs and neighborhoods, is downloaded from a website.
Second, the data is pre-processed into a dataframe. Third, coordinates are extracted for each neighborhood using API calls.
Fourth, explore and cluster neighborhoods in Toronto. Last, visualize neighborhoods and clusters on a map.*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
!pip install lxml

     |████████████████████████████████| 5.8MB 3.1MB/s eta 0:00:01


In [2]:
# step 1: download data from website
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
df = pd.read_html(url)
print(df[0].head(10))

# save table back to csv for future use
file = 'list_of_postal_codes_canada_csv'
df[0].to_csv(file, index=False)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned


# Question 1 Transform the data in the table into proper dataframe

In [3]:
# step 2: process the data into a desired dataframe
df_pc = pd.read_csv(file)

# set the column names, remove Borough='Not assigned', and copy Borough name to Neighborhood='Not assigned'
df_pc.columns = ['PostalCode', 'Borough','Neighborhood']
df_pc = df_pc[df_pc.Borough!='Not assigned']
df_pc.loc[df_pc['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df_pc['Borough']

# merge neighborhoods with the same PostalCode (assuming same Borough as well)
df_merged = df_pc.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)
df_pc = df_merged.reset_index()

df_pc.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [4]:
# print the shape of dataframe of postalcodes
print(df_pc.shape)

(103, 3)


# Question 2 Create the dataframe that has the coordinates for each neighborhood

In [6]:
# step 3: use Geocoder or csv to extract coordinates and add to the dataframe
!pip install geocoder
import geocoder

# attempt to obtain coordinates through Geocoder unsuccessful, use csv file instead
url_geo = 'http://cocl.us/Geospatial_data'
df_latlon = pd.read_csv(url_geo)

df_latlon.head()

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
     |████████████████████████████████| 829kB 33.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 23.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# clean coordinate data and merge into the previous dataframe to create a new dataframe
df_latlon.columns = ['PostalCode', 'Latitude', 'Longitude']
df_new = pd.merge(df_pc, df_latlon, on='PostalCode')

df_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


# Question 3 Explore and cluster the neighborhoods in Toronto

In [8]:
# step 4: explore and cluster neighborhoods in Toronto
# filter on Borough to get list of boroughs that contain "Toronto"
df_to = df_new[df_new.Borough.str.contains('Toronto')].reset_index(drop=True)

# drop column PostalCode as it is no longer required
df_to = df_to.drop('PostalCode', axis=1)

df_to

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,East Toronto,The Beaches,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,Downtown Toronto,Christie,43.669542,-79.422564
7,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [9]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

*We replicate some of the same analysis we did to the New York City data. First, we plot a map to visualize the city of Toronto and its neighborhoods. Second, we get nearby venues for each neighborhood by making API calls to Foursquare. Third, we group the results by neighborhood and one-hot all venue categories and calculate the mean of the frequency of occurrences of each venue category. Fourth, we run k-means to cluster the neighborhood to five clusters. Finally, we show the neighborhoods and clusters on the map.*

In [10]:
import json # library to handle JSON files
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_to = location.latitude
longitude_to = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude_to, longitude_to))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_to, longitude_to], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
# @hidden_cell
CLIENT_ID = 'D4HWMP1ULFHF5WXNORH44OCY4Y52HDARDHYDZLSHE4YF0IVB' # your Foursquare ID
CLIENT_SECRET = 'N4SEJSBFQBEABJFBBTUI5PAM2NRT24E2TFM3HAUEBJE10VI5' # your Foursquare Secret
VERSION = '20190914' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D4HWMP1ULFHF5WXNORH44OCY4Y52HDARDHYDZLSHE4YF0IVB
CLIENT_SECRET:N4SEJSBFQBEABJFBBTUI5PAM2NRT24E2TFM3HAUEBJE10VI5


In [13]:
# define a function that get nearby venues for each neighborhood, one by one
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# get nearby venues for each neighborhood in Toronto
LIMIT = 50
toronto_venues = getNearbyVenues(names=df_to['Neighborhood'], latitudes=df_to['Latitude'], longitudes=df_to['Longitude'])

toronto_venues

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
1158,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1159,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1160,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1161,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


In [50]:
toronto_venues.groupby('Neighborhood', sort=False).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Harbourfront, Regent Park",49,49,49,49,49,49
"Ryerson, Garden District",50,50,50,50,50,50
St. James Town,50,50,50,50,50,50
The Beaches,4,4,4,4,4,4
Berczy Park,50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
"Adelaide, King, Richmond",50,50,50,50,50,50
"Dovercourt Village, Dufferin",16,16,16,16,16,16


In [67]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood', sort=False).mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Harbourfront, Regent Park",0.020408,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.020408,...,0.040816,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
1,"Ryerson, Garden District",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.040000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
2,St. James Town,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
3,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.250000,0.00,0.000000,0.000000,0.0000,0.00,0.00
4,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
5,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
6,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
7,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
8,"Dovercourt Village, Dufferin",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
9,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.02,0.000000,0.000000,0.0000,0.00,0.00


In [68]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 3, 3, 3, 3, 0, 3, 0, 0, 3, 3, 3, 0, 3, 3, 0, 0, 4, 1, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 4, 3, 3, 0, 3, 3], dtype=int32)

In [69]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_to

# merge toronto_grouped with df_to to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_grouped.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the Cluster Label!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,0.020408,0.0,0.0,0.0,0.0,...,0.040816,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
1,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,0.000000,0.0,0.0,0.0,0.0,...,0.040000,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.0
2,Downtown Toronto,St. James Town,43.651494,-79.375418,3,0.000000,0.0,0.0,0.0,0.0,...,0.020000,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
3,East Toronto,The Beaches,43.676357,-79.293031,3,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.25,0.00,0.00,0.0,0.0,0.0,0.0
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.0
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,0.020000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.0
6,Downtown Toronto,Christie,43.669542,-79.422564,3,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
7,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.0
8,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,3,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.02,0.00,0.0,0.0,0.0,0.0


In [70]:
# step 5: show neighborhoods and clusters on the map
map_clusters = folium.Map(location=[latitude_to, longitude_to], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters